#### 필요한 라이브러리 불러오기 

In [ ]:
from keras.models import load_model
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras import optimizers
import numpy as np
import tensorflow_addons as tfa

#### 모델 함수

In [ ]:
# 인자로 전처리된 학습데이터의 경로, 학습횟수(EPOCH), 배치사이즈(BATCH_SIZE), 옵티마이저 종류(OPTIMIZER), 
# 학습률(LEARNING_RATE), 비용함수 종류(COST), 불러올 모델의 경로(MODEL_LOAD_PATH), 학습한 모델을 저장할 경로(MODEL_SAVE_PATH)를 
# 인자로 받음

def modeling(TRAIN_PATH, EPOCH, BATCH_SIZE,LEARNING_RATE, COST, MODEL_LOAD_PATH, MODEL_SAVE_PATH, OPTIMIZER):
    
    # 옵티마이저 선택지 (벡엔드에는 string 형식으로만 인자를 받을 수 있기 때문에)
    optimizer_dic = {'RMSprop': optimizers.RMSprop, 'Adam' : optimizers.Adam}
    
    # 뭔가 잘못 입력되면 그냥 Adam을 optimizer로 사용할 수 있도록 
    if ((OPTIMIZER!='Adam') or (OPTIMIZER!='RMSprops')):
        OPTIMIZER = 'Adam'
    
    print(OPTIMIZER)
    # 전이학습모델 불러오기 
    model = load_model(MODEL_LOAD_PATH)
    print("모델을 로드했습니다.")
    
    # model compile (optimizer, learning rate, cost function 변경 가능)
    model.compile(loss= COST,
                  optimizer= optimizer_dic[OPTIMIZER](learning_rate = LEARNING_RATE),
              metrics=[tf.keras.metrics.Precision(name='precision')\
                          ,tf.keras.metrics.Recall(name='recall')\
                          ,tf.keras.metrics.FalsePositives(name='false_positives')\
                          ,tf.keras.metrics.FalseNegatives(name='false_negatives')\
                          ,tfa.metrics.FBetaScore(num_classes=2,average="micro",threshold=0.5)
                          ,'acc'])
    
    # lr : 학습률 줄여나가기 
    lr = ReduceLROnPlateau(monitor="val_loss", factor=0.9, patience=6, verbose=1)
    
    # es: 더 이상 validation loss가 줄어들지 않으면 멈추기
    es = EarlyStopping(monitor="val_loss", patience=10, verbose=1, mode="min", restore_best_weights=True)
    
    # 모델에 넣을 데이터 불러오기 
    accept_list = [TRAIN_PATH+'/ACCEPT/'+i for i in os.listdir(TRAIN_PATH+'/ACCEPT') if i.split('.')[-1]=="npz"]
    reject_list = [TRAIN_PATH+'/REJECT/'+i for i in os.listdir(TRAIN_PATH+'/REJECT') if i.split('.')[-1]=="npz"]
    
    # accept 이미지들 받기 
    accept_images = []
    for idx,path in enumerate(accept_list):
        accept_npz = np.load(path, allow_pickle=True)
        accept_images.append(accept_npz['img'])
    # 이중 리스트 풀기 
    accept_images = np.concatenate(accept_images).tolist()
    
    # reject 이미지들 받기 
    reject_images = []
    for idx,path in enumerate(reject_list):
        reject_npz = np.load(path, allow_pickle=True)
        reject_images.append(reject_npz['img'])
    # 이중 리스트 풀기
    reject_images = np.concatenate(reject_images).tolist()
    
    # accept, reject 각각 라벨 생성
    accept_label = [[1,0]]*len(accept_images)
    reject_label = [[0,1]]*len(reject_images)
    
    # 각각 X랑 Y로 합치기 
    X = accept_images + reject_images
    Y = accept_label + reject_label 
    

    # X와 Y를 train 데이터와 validation 데이터로 나누기 
    tf.compat.v1.random.set_random_seed(777) 
    train_x, val_x, train_y, val_y = train_test_split(X, Y, test_size=0.2, random_state=777, shuffle=True, stratify=Y)
    
    # 학습시작
    print("학습을 시작합니다.")
    history = model.fit(np.array(train_x).reshape(-1,400,400,1), np.array(train_y), 
                        epochs=EPOCH, verbose=0,callbacks=[es,lr],
                        batch_size=BATCH_SIZE, validation_data = (np.array(val_x).reshape(-1,400,400,1),np.array(val_y)))
    
    # 학습된 모델을 지정 경로로 저장 
    model.save(MODEL_SAVE_PATH)
    
    # 벡엔드에서 출력값을 파싱하기 편하도록 함수 만들기
    # ***.**** 으로 소숫점 앞은 세자리까지, 소숫점 뒤는 네자리까지 표기 하도록 
    def make_output(number):
        if(number>=1000):
            number = '999.0000'
        else:
            if(number//10==0):
                number = '00'+"{:.4f}".format(number)
            elif(number//100==0):
                number = '0'+"{:.4f}".format(number)
            else:
                number = "{:.4f}".format(number)
        return number    
    
    # 모든 평가지표에 앞서 만든 함수를 적용하고 변수에 값을 할당하기 
    for i in range(len(history.history['loss'])):
        loss = make_output(history.history['loss'][i])
        precision = make_output(history.history['precision'][i])
        recall = make_output(history.history['recall'][i])
        false_positives = make_output(history.history['false_positives'][i])
        false_negatives = make_output(history.history['false_negatives'][i])
        accuracy = make_output(history.history['acc'][i])
        f1_score = make_output(history.history['fbeta_score'][i])
        
        # 벡엔드에서 파싱해서 값을 프론트로 넘겨줄 수 있도록 출력하기 
        print("loss:", loss, "accuracy:", accuracy, "recall:", recall, "precision:", precision, "false_positives:", false_positives, 
             "false_negatives:", false_negatives, "f1_score:", f1_score)